In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import roc_auc_score
import mlflow
import mlflow.tensorflow

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [4]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=3, strides=2, padding="same", input_shape=(28, 28, 1), activation="relu"))
model.add(Conv2D(filters=8, kernel_size=3, strides=2, padding="same", activation="relu"))
model.add(Flatten())
model.add(Dense(30, activation="relu"))
model.add(Dense(20, activation="relu"))
model.add(Dense(10, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 8)           1160      
_________________________________________________________________
flatten (Flatten)            (None, 392)               0         
_________________________________________________________________
dense (Dense)                (None, 30)                11790     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
Total params: 13,940
Trainable params: 13,940
Non-trainable params: 0
____________________________________________________

In [5]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [6]:
mlflow.set_experiment("TF_Keras_MNIST")

with mlflow.start_run():
    mlflow.tensorflow.autolog()  # tensorflow.keras와 관련된 모든 파라미터와 metric을 기록

    model.fit(x_train, y_train, batch_size=256, epochs=10)
    preds = model.predict(x_test)
    preds = np.round(preds)

    eval_acc = model.evaluate(x_test, y_test)[1]
    auc_score = roc_auc_score(y_test, preds)

    print("eval_acc :", eval_acc)
    print("auc_score :", auc_score)

    mlflow.tensorflow.mlflow.log_metric("eval_acc", eval_acc)
    mlflow.tensorflow.mlflow.log_metric("auc_score", auc_score)

mlflow.end_run()

2022/03/11 13:42:44 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Epoch 1/10
235/235 [==============================] - 4s 14ms/step - loss: 1.0710 - accuracy: 0.6989
Epoch 2/10
235/235 [==============================] - 4s 16ms/step - loss: 0.2339 - accuracy: 0.9322
Epoch 3/10
235/235 [==============================] - 3s 14ms/step - loss: 0.1479 - accuracy: 0.9572
Epoch 4/10
235/235 [==============================] - 5s 23ms/step - loss: 0.1082 - accuracy: 0.9683
Epoch 5/10
235/235 [==============================] - 6s 26ms/step - loss: 0.0866 - accuracy: 0.9741
Epoch 6/10
235/235 [==============================] - 6s 27ms/step - loss: 0.0718 - accuracy: 0.9778
Epoch 7/10
235/235 [==============================] - 7s 28ms/step - loss: 0.0617 - accuracy: 0.9813
Epoch 8/10
235/235 [==============================] - 6s 26ms/step - loss: 0.0548 - accuracy: 0.9832
Epoch 9/10
235/235 [==============================] - 7s 28ms/step - loss: 0.0469 - accuracy: 0.9855
Epoch 10/10
235/235 [==============================] - 6s 28ms/step - loss: 0.0401 - accura

In [7]:
loaded_model = mlflow.keras.load_model("runs:/ad35de0f8b69419385b5f0ec4a726d35/model")  # keras로 작성한 모델이므로 mlflow.tensorflow 대신 mlflow.keras로 호출

In [8]:
eval_loss, eval_acc = loaded_model.evaluate(x_test, y_test)
preds = loaded_model.predict(x_test)
preds = np.round(preds)

eval_auc = roc_auc_score(y_test, preds)

print("Eval Loss :", eval_loss)
print("Eval Acc :", eval_acc)
print("Eval AUC :", eval_auc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0844 - accuracy: 0.9751
Eval Loss : 0.08443881571292877
Eval Acc : 0.9750999808311462
Eval AUC : 0.9853014921721911
